In [1]:
import pandas as pd
import numpy as np
import os
print os.getcwd()
dir = "/Users/walterdempsey/Box/MD2K Processed Data/smoking-lvm-cleaned-data/"
os.chdir(dir)
os.getcwd()

/Users/walterdempsey/Documents/github/sense2stop-lvm-wd/data_summary


'/Users/walterdempsey/Box/MD2K Processed Data/smoking-lvm-cleaned-data'

In [5]:
# read data
original_data = pd.read_csv(dir+'eventcontingent-ema.csv')
original_data = original_data.drop(['offset'], axis=1)
backup_data = pd.read_csv(dir+'eventcontingent-ema-backup.csv')

In [6]:
# check data types
print(original_data.dtypes)
print("")
print(backup_data.dtypes)

status               object
smoke                object
when_smoke           object
urge                float64
cheerful            float64
happy               float64
angry               float64
stress              float64
sad                 float64
see_or_smell        float64
access              float64
smoking_location    float64
participant_id        int64
timestamp             int64
date                 object
hour                  int64
minute                int64
day_of_week          object
dtype: object

status               object
smoke                object
when_smoke           object
urge                  int64
cheerful              int64
happy                 int64
angry                 int64
stress                int64
sad                   int64
see_or_smell          int64
access                int64
smoking_location      int64
participant_id        int64
timestamp           float64
date                 object
hour                  int64
minute                int64
day_o

In [7]:
# convert data types to be consistent
backup_data['urge'] = backup_data['urge'].astype(float)
backup_data['cheerful'] = backup_data['cheerful'].astype(float)
backup_data['happy'] = backup_data['happy'].astype(float)
backup_data['angry'] = backup_data['angry'].astype(float)
backup_data['stress'] = backup_data['stress'].astype(float)
backup_data['sad'] = backup_data['sad'].astype(float)
backup_data['access'] = backup_data['access'].astype(float)
backup_data['see_or_smell'] = backup_data['see_or_smell'].astype(float)
backup_data['smoking_location'] = backup_data['urge'].astype(float)

print(original_data.dtypes)
print(backup_data.dtypes)

status               object
smoke                object
when_smoke           object
urge                float64
cheerful            float64
happy               float64
angry               float64
stress              float64
sad                 float64
see_or_smell        float64
access              float64
smoking_location    float64
participant_id        int64
timestamp             int64
date                 object
hour                  int64
minute                int64
day_of_week          object
dtype: object
status               object
smoke                object
when_smoke           object
urge                float64
cheerful            float64
happy               float64
angry               float64
stress              float64
sad                 float64
see_or_smell        float64
access              float64
smoking_location    float64
participant_id        int64
timestamp           float64
date                 object
hour                  int64
minute                int64
day_of

In [8]:
# check how many entries for each participant
print(original_data.shape)
original_data['participant_id'].value_counts().sort_index()

(325, 18)


201     4
202     9
205     6
208     7
209     4
211     7
212     3
213     7
214    11
215     6
216     2
217     8
218     5
219    23
220     9
221     3
222     5
228    18
229    13
230     4
231     6
233     9
234     4
235     6
237     1
238    19
239     1
240    31
241     2
242     7
243     7
244     5
245     5
246     6
247     1
248     6
249     7
250     7
251     7
252    14
253    20
Name: participant_id, dtype: int64

In [9]:
print(backup_data.shape)
backup_data['participant_id'].value_counts().sort_index()

(318, 33)


202     9
204     5
205     6
207     9
208    14
209     4
211     7
212     3
213     7
214    11
215     6
216     2
217     8
218     5
219    23
220    10
222     8
228    18
230     4
231     6
233     9
234     4
235     6
237     1
238    19
239     1
240    31
241     2
242     7
243     7
245     5
246     6
247     1
248     6
249     7
250     7
252    14
253    20
Name: participant_id, dtype: int64

In [10]:
# Look at the participant_id intersection of the backup and original datasets
unique_backup_ids = set(backup_data.participant_id)
unique_original_ids = set(original_data.participant_id)
ids_intersection = unique_backup_ids.intersection(unique_original_ids)

original_data_subset = original_data[original_data.participant_id.isin(ids_intersection)]
backup_data_subset = backup_data[backup_data.participant_id.isin(ids_intersection)]

print unique_backup_ids.difference(unique_original_ids)
print unique_original_ids.difference(unique_backup_ids)


set([204, 207])
set([201, 251, 244, 221, 229])


In [11]:
# In intersection, the backup is a superset of the observations. 
# Therefore, the backup shoud be used for analysis where possible.
print(original_data_subset.shape)
temp_original = pd.DataFrame(original_data_subset['participant_id'].value_counts().sort_index())
temp_backup = np.array(backup_data_subset['participant_id'].value_counts().sort_index())
temp_original['backup'] = temp_backup
temp_original.columns = ['original', 'backup']
temp_original

(293, 18)


,original,backup
202,9,9
205,6,6
208,7,14
209,4,4
211,7,7
212,3,3
213,7,7
214,11,11
215,6,6
216,2,2


In [12]:
s = set()
d = {}

lst2 = []

# store entries in the backup data
for index, row in backup_data.iterrows():
    participant_id = row['participant_id']
    hour = row['hour']
    minute = row['minute']
    day_of_week = row['day_of_week']
    valid_key = (participant_id, hour, minute, day_of_week)
    s.add(valid_key)
    d[valid_key] = index

# store entries in the original data
for index, row in original_data.iterrows():
    participant_id = row['participant_id']
    hour = row['hour']
    minute = row['minute']
    day_of_week = row['day_of_week']
    valid_key = (participant_id, hour, minute, day_of_week)
    # remove if also in the backup data
    if valid_key in s:
        s.remove(valid_key)
    else:
        lst2.append(index)

lst = []
for i in s:
    lst.append(d[i])

lst.sort()
difference = backup_data.ix[lst] 
print(difference.shape)
difference # rows that are only in the backup but not in the original

(25, 33)


/Library/Python/2.7/site-packages/ipykernel_launcher.py:34: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


,status,smoke,when_smoke,urge,cheerful,happy,angry,stress,sad,see_or_smell,...,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32
9,COMPLETED,Yes,Less than 5 minutes,6.0,5.0,5.0,2.0,2.0,2.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,COMPLETED,Yes,Less than 5 minutes,4.0,3.0,3.0,3.0,5.0,3.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,COMPLETED,Yes,Less than 5 minutes,5.0,4.0,4.0,3.0,4.0,3.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,COMPLETED,Yes,Less than 5 minutes,4.0,4.0,4.0,3.0,3.0,3.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,COMPLETED,Yes,15 to 30 minutes,3.0,3.0,3.0,3.0,4.0,3.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,COMPLETED,Yes,Less than 5 minutes,5.0,3.0,4.0,2.0,2.0,2.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,COMPLETED,Yes,5 to 15 minutes,4.0,5.0,5.0,1.0,3.0,1.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,COMPLETED,Yes,Less than 5 minutes,3.0,5.0,5.0,1.0,3.0,1.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,COMPLETED,Yes,5 to 15 minutes,4.0,5.0,5.0,2.0,2.0,2.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,COMPLETED,Yes,Less than 5 minutes,4.0,4.0,4.0,2.0,2.0,2.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
difference = original_data.ix[lst2]
print(difference.shape)
difference # rows in the original data but not in the backup data

(32, 18)


/Library/Python/2.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


,status,smoke,when_smoke,urge,cheerful,happy,angry,stress,sad,see_or_smell,access,smoking_location,participant_id,timestamp,date,hour,minute,day_of_week
0,COMPLETED,Yes,Less than 5 minutes,6.0,4.0,4.0,3.0,3.0,3.0,3.0,4.0,3.0,201,1498258560862,2017-06-23 18:56:00,18,56,Friday
1,COMPLETED,Yes,5 to 15 minutes,4.0,4.0,4.0,3.0,2.0,1.0,3.0,4.0,4.0,201,1498264205611,2017-06-23 20:30:05,20,30,Friday
2,COMPLETED,Yes,Less than 5 minutes,5.0,4.0,3.0,4.0,4.0,4.0,3.0,4.0,4.0,201,1498331842758,2017-06-24 15:17:22,15,17,Saturday
3,COMPLETED,Yes,Less than 5 minutes,4.0,4.0,4.0,3.0,3.0,3.0,3.0,4.0,4.0,201,1498345697395,2017-06-24 19:08:17,19,8,Saturday
111,COMPLETED,Yes,Less than 5 minutes,4.0,4.0,4.0,4.0,2.0,2.0,3.0,4.0,3.0,221,1511819188581,2017-11-27 16:46:28,16,46,Monday
112,COMPLETED,Yes,Less than 5 minutes,5.0,2.0,2.0,5.0,6.0,4.0,3.0,4.0,4.0,221,1511914895469,2017-11-28 19:21:35,19,21,Tuesday
113,COMPLETED,Yes,Less than 5 minutes,4.0,2.0,2.0,6.0,6.0,5.0,3.0,4.0,4.0,221,1511919761132,2017-11-28 20:42:41,20,42,Tuesday
137,COMPLETED,Yes,5 to 15 minutes,4.0,4.0,4.0,3.0,3.0,3.0,4.0,4.0,4.0,229,1523645597416,2018-04-13 14:53:17,14,53,Friday
138,COMPLETED,Yes,5 to 15 minutes,4.0,4.0,4.0,3.0,3.0,3.0,4.0,4.0,4.0,229,1523660369486,2018-04-13 18:59:29,18,59,Friday
139,COMPLETED,Yes,5 to 15 minutes,4.0,4.0,4.0,3.0,3.0,3.0,4.0,4.0,4.0,229,1523666894137,2018-04-13 20:48:14,20,48,Friday
